# LLVM Cauldron - Wuthering Bytes 2016-09-08

# Generating Python & Ruby bindings from C++

### Jonathan B Coe
### jbcoe@me.com

## https://github.com/jbcoe/C_API_generation
## https://github.com/jbcoe/clang_cpp_code_model

Write a C++ class out to a file in the current working directory

In [1]:
outputfile = "Shape.h"

In [2]:
%%file $outputfile
#include <stdexcept>
#include <string>

struct Shape
{
  virtual ~Shape() = default;
  virtual double area() const = 0;
  virtual double perimeter() const = 0;
  virtual const char* name() const = 0;
} __attribute__((annotate("GENERATE_C_API")));

static const double pi = 4.0;

class Circle : public Shape
{
  const double radius_;

public:
  double area() const override
  {
    return pi * radius_ * radius_;
  }

  double perimeter() const override
  {
    return 2 * pi * radius_;
  }

  const char* name() const override
  {
    return "Circle";
  }

  Circle(double radius) : radius_(radius)
  {
    if ( radius < 0 ) 
    { 
      std::string s = "Circle radius \"" + std::to_string(radius_) + "\" must be non-negative.";
      throw std::runtime_error(s);
    }
  }
};

Overwriting Shape.h


Compile our header to check it's valid C++

In [3]:
%%sh
clang++-3.8 -x c++ -fsyntax-only -std=c++14 Shape.h 

Read the code using libclang

In [4]:
import clang.cindex

index = clang.cindex.Index.create()
translation_unit = index.parse(outputfile, ['-x', 'c++', '-std=c++14'])

In [5]:
import asciitree

def node_children(node):
    return (c for c in node.get_children() if c.location.file.name == outputfile)

print asciitree.draw_tree(translation_unit.cursor,
  lambda n: [c for c in node_children(n)],
  lambda n: "%s (%s)" % (n.spelling or n.displayname, str(n.kind).split(".")[1]))


Shape.h (TRANSLATION_UNIT)
  +--Shape (STRUCT_DECL)
  |  +--GENERATE_C_API (ANNOTATE_ATTR)
  |  +--~Shape (DESTRUCTOR)
  |  |  +-- (COMPOUND_STMT)
  |  +--area (CXX_METHOD)
  |  +--perimeter (CXX_METHOD)
  |  +--name (CXX_METHOD)
  +--pi (VAR_DECL)
  |  +-- (FLOATING_LITERAL)
  +--Circle (CLASS_DECL)
     +--struct Shape (CXX_BASE_SPECIFIER)
     |  +--struct Shape (TYPE_REF)
     +--radius_ (FIELD_DECL)
     +-- (CXX_ACCESS_SPEC_DECL)
     +--area (CXX_METHOD)
     |  +-- (CXX_OVERRIDE_ATTR)
     |  +-- (COMPOUND_STMT)
     |     +-- (RETURN_STMT)
     |        +-- (BINARY_OPERATOR)
     |           +-- (BINARY_OPERATOR)
     |           |  +--pi (UNEXPOSED_EXPR)
     |           |  |  +--pi (DECL_REF_EXPR)
     |           |  +--radius_ (UNEXPOSED_EXPR)
     |           |     +--radius_ (MEMBER_REF_EXPR)
     |           +--radius_ (UNEXPOSED_EXPR)
     |              +--radius_ (MEMBER_REF_EXPR)
     +--perimeter (CXX_METHOD)
     |  +-- (CXX_OVERRIDE_ATTR)
     |  +-- (COMPOUND_STM

Turn the AST into some easy to manipulate Python classes

In [6]:
import sys
sys.path.insert(0,'../externals/clang_cpp_code_model')
import cppmodel

In [7]:
model = cppmodel.Model(translation_unit)

In [8]:
[f.name for f in model.functions][-5:]

['to_wstring', 'operator""s', 'operator""s', 'operator""s', 'operator""s']

In [9]:
[c.name for c in model.classes][-5:]

['__basic_string_common', 'basic_string', 'basic_string', 'Shape', 'Circle']

In [10]:
shape_class = [c for c in model.classes if c.name=='Shape'][0]

In [11]:
["{}::{}".format(shape_class.name,m.name) for m in shape_class.methods]

['Shape::area', 'Shape::perimeter', 'Shape::name']

Look at the templates the generator uses

In [12]:
%cat ../templates/json.tmpl

{
  "name" : "{{class.name}}"{% if class.methods %},
  "methods" : [{% for method in class.methods %}
    {
      "name" : "{{method.name}}",
      "return_type" : "{{method.return_type}}"
    }{% if not forloop.last %},{% endif %}{% endfor %}
  ]{% endif %}
}


Run the code generator

In [13]:
%%sh
export PYTHONPATH=../externals/clang_cpp_code_model:$PYTHONPATH
../generate.py Shape.h ../templates/ . 

See what it created

In [14]:
%ls

$outputfile                Shape.pyc
CMakeCache.txt             Shape.rb
CMakeFiles/                Shape_c.cpp
CMakeLists.txt             Shape_c.h
LLVM-Cauldron.ipynb        Shape_cpp.h
LLVM-Cauldron.slides.html  Shape_mocks.h
Makefile                   cmake_install.cmake
Shape.h                    libShape_c.dylib*
Shape.json                 slideshow.sh*
Shape.py


In [15]:
%cat Shape.json

{
  "name" : "Shape",
  "methods" : [
    {
      "name" : "area",
      "return_type" : "double"
    },
    {
      "name" : "perimeter",
      "return_type" : "double"
    },
    {
      "name" : "name",
      "return_type" : "const char *"
    }
  ]
}


Build some bindings with the generated code.

In [16]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
set(CMAKE_CXX_COMPILER /usr/local/bin/clang++-3.8)
add_library(Shape_c SHARED Shape_c.cpp)

Overwriting CMakeLists.txt


In [17]:
%%sh
cmake . 
cmake --build .

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/jon/DEV/C_APICodeGen/demos
Scanning dependencies of target Shape_c
[ 50%] Building CXX object CMakeFiles/Shape_c.dir/Shape_c.cpp.o
[100%] Linking CXX shared library libShape_c.dylib
[100%] Built target Shape_c


In [19]:
%%python2
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print "A {} with radius {} has area {}".format(c.name(), 8, c.area())

A Circle with radius 8 has area 256.0


In [20]:
%%script pypy
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print "A {} with radius {} has area {}".format(c.name(), 8, c.area())

A Circle with radius 8 has area 256.0


In [21]:
%%ruby
load "Shape.rb"
c = Circle.new(8)

puts("A #{c.name()} with radius #{8} has area #{c.area()}")

A Circle with radius 8 has area 256.0
